In [1]:
import pandas as pd
from pymongo import MongoClient
import re
import numpy as np

client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [2]:
companies = db.companies.find({
    "offices":{
        "$not":{
            "$size":0
        }
    }
, "founded_year":{
    "$gte": 2007
}
}
,{"name":1, "founded_year":1, "offices":1, "category_code":1, "total_money_raised":1, "deadpooled_year":1, "number_of_employees":1})

In [3]:
data_companies = pd.DataFrame(companies)
data_companies.head()

,_id,category_code,deadpooled_year,founded_year,name,number_of_employees,offices,total_money_raised
0,52cdef7c4bab8bd675297d97,news,NaN,2007,Scribd,50.0,"[{'description': 'HQ', 'address1': '539 Bryant...",$25.8M
1,52cdef7c4bab8bd675297d9c,social,NaN,2007,MeetMoi,15.0,"[{'description': None, 'address1': '', 'addres...",$5.58M
2,52cdef7c4bab8bd675297da0,games_video,2013.0,2007,Babelgum,NaN,"[{'description': '', 'address1': '', 'address2...",$13.2M
3,52cdef7c4bab8bd675297daa,games_video,2008.0,2007,Sparter,NaN,"[{'description': None, 'address1': None, 'addr...",$0
4,52cdef7c4bab8bd675297da9,web,NaN,2007,Mahalo,40.0,"[{'description': '', 'address1': '3525 Eastham...",$21M


In [4]:
def getFirst(data):
    data = data['offices']
    principal = None
    if data[0]['latitude'] and data[0]['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }

    return {
        "lat": data[0]['latitude'],
        "lng": data[0]['longitude'],
        "principal_office": principal
    }


In [5]:
geo_office = data_companies[["offices"]].apply(getFirst, result_type="expand", axis=1)

In [6]:
df_clean = pd.concat([data_companies,geo_office], axis=1)[["principal_office"]]

In [7]:
data_companies['geoDescription'] = df_clean

In [8]:
data_companies.head()

,_id,category_code,deadpooled_year,founded_year,name,number_of_employees,offices,total_money_raised,geoDescription
0,52cdef7c4bab8bd675297d97,news,NaN,2007,Scribd,50.0,"[{'description': 'HQ', 'address1': '539 Bryant...",$25.8M,"{'type': 'Point', 'coordinates': [-122.404052,..."
1,52cdef7c4bab8bd675297d9c,social,NaN,2007,MeetMoi,15.0,"[{'description': None, 'address1': '', 'addres...",$5.58M,"{'type': 'Point', 'coordinates': [-73.985506, ..."
2,52cdef7c4bab8bd675297da0,games_video,2013.0,2007,Babelgum,NaN,"[{'description': '', 'address1': '', 'address2...",$13.2M,"{'type': 'Point', 'coordinates': [-6.267494, 5..."
3,52cdef7c4bab8bd675297daa,games_video,2008.0,2007,Sparter,NaN,"[{'description': None, 'address1': None, 'addr...",$0,"{'type': 'Point', 'coordinates': [-95.712891, ..."
4,52cdef7c4bab8bd675297da9,web,NaN,2007,Mahalo,40.0,"[{'description': '', 'address1': '3525 Eastham...",$21M,"{'type': 'Point', 'coordinates': [-118.487267,..."


In [9]:
data_companies['category_code'].value_counts()

web                 1167
software             514
games_video          365
advertising          287
mobile               277
other                222
ecommerce            222
enterprise           213
network_hosting      160
public_relations     154
consulting           141
search               115
cleantech             61
biotech               53
hardware              37
social                29
analytics             29
security              24
education             19
news                  18
music                 18
finance               16
travel                14
photo_video           14
messaging             12
health                 8
sports                 8
semiconductor          6
legal                  6
medical                6
fashion                5
hospitality            4
real_estate            4
design                 3
manufacturing          3
transportation         1
nonprofit              1
nanotech               1
automotive             1
Name: category_code, dtyp

In [10]:
list_category = {'web': 'tech', 'software': 'tech', 
                 'games_video':'tech', 'advertising':'tech', 'mobile':'tech', 'ecommerce':'tech', 'search':'tech', 'network_hosting':'tech'
                ,'consulting':'tech','hardware':'tech', 'biotech':'tech', 'cleantech':'tech', 'analytics':'tech','hardware':'tech', 'photo_video':'tech',
                'messaging':'tech', 'design':'tech'}

In [11]:
data_companies = data_companies.replace(to_replace=list_category, inplace=False)

In [12]:
data_companies['founded_year'] = data_companies['founded_year'].dropna(axis=0).astype('int64', inplace=True)

In [13]:
latitude = [d[0].get('latitude') for d in data_companies.offices]
longitude = [d[0].get('longitude') for d in data_companies.offices]

In [14]:
data_companies['latitude'] = latitude
data_companies['longitude'] = longitude

In [15]:
city = [d[0].get('city') for d in data_companies.offices]
country = [d[0].get('country_code') for d in data_companies.offices]

In [16]:
data_companies['city'] = city
data_companies['country'] = country

In [17]:
data_companies.head()

,_id,category_code,deadpooled_year,founded_year,name,number_of_employees,offices,total_money_raised,geoDescription,latitude,longitude,city,country
0,52cdef7c4bab8bd675297d97,news,NaN,2007,Scribd,50.0,"[{'description': 'HQ', 'address1': '539 Bryant...",$25.8M,"{'type': 'Point', 'coordinates': [-122.404052,...",37.789634,-122.404052,San Francisco,USA
1,52cdef7c4bab8bd675297d9c,social,NaN,2007,MeetMoi,15.0,"[{'description': None, 'address1': '', 'addres...",$5.58M,"{'type': 'Point', 'coordinates': [-73.985506, ...",40.757929,-73.985506,New York City,USA
2,52cdef7c4bab8bd675297da0,tech,2013.0,2007,Babelgum,NaN,"[{'description': '', 'address1': '', 'address2...",$13.2M,"{'type': 'Point', 'coordinates': [-6.267494, 5...",53.344104,-6.267494,London,GBR
3,52cdef7c4bab8bd675297daa,tech,2008.0,2007,Sparter,NaN,"[{'description': None, 'address1': None, 'addr...",$0,"{'type': 'Point', 'coordinates': [-95.712891, ...",37.090240,-95.712891,None,USA
4,52cdef7c4bab8bd675297da9,tech,NaN,2007,Mahalo,40.0,"[{'description': '', 'address1': '3525 Eastham...",$21M,"{'type': 'Point', 'coordinates': [-118.487267,...",34.017606,-118.487267,Culver City,USA


In [18]:
def dropcolumns(data, columns):
        data = data.drop(columns, axis=1)
        return data

In [19]:
data_companies = dropcolumns(data_companies, ['offices', '_id'])

In [20]:
data_companies = data_companies[data_companies['deadpooled_year'].isna()]

In [21]:
data_companies['city'].value_counts()

San Francisco       277
New York            221
                    203
London              147
Seattle              69
Palo Alto            51
Los Angeles          50
Paris                45
Toronto              42
Chicago              39
Berlin               39
Austin               39
Cambridge            37
San Jose             33
Bangalore            32
Mountain View        30
Atlanta              28
San Diego            28
Santa Monica         27
Boston               27
Vancouver            27
Amsterdam            27
Barcelona            24
Sunnyvale            23
Madrid               21
San Mateo            20
Portland             18
Menlo Park           18
Singapore            18
Philadelphia         17
                   ... 
Belen                 1
Warminster            1
Itasca                1
Acton                 1
Richmond  Rhode       1
QuÃ©bec               1
Rishpon               1
Washington, D.C.      1
Goshen                1
Ripalimosani          1
V. N. Gaia      

In [22]:
def cleanLocation(location):
    location = str(location)
    if (re.search('[Ll]os [Aa]ngeles', location)):
        return 'Los Angeles'
    elif (re.search('IRVINGTON', location)):
          return 'Irvington'
    else:
        return location

In [23]:
data_companies['city'] = data_companies['city'].apply(cleanLocation)

In [24]:
data_companies.dropna(subset=['founded_year', 'latitude', 'longitude'], inplace=True)

In [25]:
data_companies = dropcolumns(data_companies, 'deadpooled_year')

In [26]:
def moneyRaise(value):
    dicc_coin = {'CAD': 1.3124497992,'RUB': 63.7561301828, 'EUR': 0.8916629514, 'GBP': 0.7991529202}
    values_money = {'K':1000, 'M':1000000, 'B': 100000000000}
    value_number = float(re.search('[+-]?([0-9]*[.])?[0-9]+', value)[0])
    if value.endswith('B'):
        exchange = value_number*(values_money['B'])
    elif value.endswith('K[k]'):
        exchange = value_number*(values_money['K'])
    elif value.endswith('M'):
        exchange = value_number*(values_money['M'])
    elif value.startswith("C"):
        exchange =  value_number*(dicc_coin['CAD'])
    elif value.startswith("$"):
        exchange =  value_number
    elif value.startswith("€"):
        exchange = value_number*(dicc_coin['EUR'])
    elif value.startswith("£"):
        exchange = value_number*(dicc_coin['GBP'])
    elif value.startswith("r"):
        exchange = value_number*(dicc_coin['RUB'])
    else:
        exchange = value_number
    return int(exchange)

In [27]:
data_companies['total_money_raised'] = data_companies['total_money_raised'].apply(moneyRaise)

In [28]:
def fillNaN(data, col):
    [data[col].fillna(0, inplace=True) for col in data.columns]
    return data

In [29]:
data_companies = fillNaN(data_companies, 'number_of_employees')

In [30]:
data_companies = data_companies[data_companies['number_of_employees'] != 0.0]
data_companies = data_companies[data_companies['total_money_raised']!=0]

In [31]:
data_companies = data_companies[data_companies['country'] != 'USA']

In [32]:
data_companies = data_companies.reset_index(drop=True)

In [33]:
data_companies.head()

,category_code,founded_year,name,number_of_employees,total_money_raised,geoDescription,latitude,longitude,city,country
0,tech,2007,Wakoopa,8.0,1000000,"{'type': 'Point', 'coordinates': [4.8948623, 5...",52.374523,4.894862,Amsterdam,NLD
1,tech,2007,boo-box,50.0,300,"{'type': 'Point', 'coordinates': [-46.679287, ...",-23.558584,-46.679287,Sao Paulo,BRA
2,finance,2007,Seedcamp,4.0,5000000,"{'type': 'Point', 'coordinates': [-0.1418973, ...",51.510880,-0.141897,London,GBR
3,tech,2007,Flixwagon,10.0,2500000,"{'type': 'Point', 'coordinates': [34.7595, 32....",32.055400,34.759500,Tel-Aviv,ISR
4,enterprise,2007,UnLtdWorld,4.0,199,"{'type': 'Point', 'coordinates': [-0.0926903, ...",51.522413,-0.092690,London,GBR


In [35]:
data_companies.to_json('./data_companies_clean.json', orient='records')